# Mounting Google Drive

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


# FUNCTION TO EXTRACT IMAGES AND APPLY DATA AUGUMENTATION

In [2]:
def img_db_creator(p1, p2):

    df_img = [] # List to hold collection of images and labels

    # Loop to add images of single root
    for img in os.listdir(p1):
        k = cv2.imread(os.path.join(p1, img), cv2.IMREAD_GRAYSCALE)
        l = cv2.resize(k, (230, 130))
        df_img.append([l, 0])

    # Flipping Images
        k1 = cv2.flip(k, 0) # flipping horizontally
        k2 = cv2.flip(k, 1) # flipping vertically

    # Resizing Flipped images
        l1 = cv2.resize(k1, (230, 130))
        l2 = cv2.resize(k2, (230, 130))

    # Adding Flipped Images
        df_img.append([l1, 0])
        df_img.append([l2, 0])

    # Loop to add images of single root
    for img in os.listdir(p2):
        k = cv2.imread(os.path.join(p2, img), cv2.IMREAD_GRAYSCALE)
        l = cv2.resize(k, (230, 130))
        df_img.append([l, 1])

        # Flipping Images
        k1 = cv2.flip(k, 0) # flipping horizontally
        k2 = cv2.flip(k, 1) # flipping vertically

        # Resizing Flipped images
        l1 = cv2.resize(k1, (230, 130))
        l2 = cv2.resize(k2, (230, 130))

       # Adding Flipped Images
        df_img.append([l1, 1])
        df_img.append([l2, 1])


    random.shuffle(df_img) # Shuffling image and label pairs for more robust model

    X_temp = [] # List to hold images
    Y_temp = [] # List to hold labels

    # Loop to extract labels and images from the variable named "df_img"
    for features, lbls in df_img:
        X_temp.append(features)
        Y_temp.append(lbls)

    X_temp = np.array(X_temp).reshape(-1, 230, 130, 1) # Converting to array and reshaping
    Y_temp = np.array(Y_temp) # Converting to array

    return X_temp, Y_temp # Returning variables to be used as primary variables

# MAIN BODY OF THE PROGRAM

## Importing Libraries and Functionalities

In [3]:
import random
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dense
from keras import backend as b
from keras.callbacks import ModelCheckpoint, CSVLogger

## Path to images

In [4]:
p_one = r"/content/drive/MyDrive/Colab Notebooks/root_canal_classification/dataset_training/one"
p_multi = r"/content/drive/MyDrive/Colab Notebooks/root_canal_classification/dataset_training/multiple"

## Creating Primary Variables

In [5]:
X, Y = img_db_creator(p_one, p_multi)

## Creating Training and Testing Variables

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

## Normalizing the Pixel Values

In [7]:
X_train = X_train/255
X_test = X_test/255

## Changing Y_train and Y_test

In [8]:
Y_train = np_utils.to_categorical(Y_train, 2)
Y_test = np_utils.to_categorical(Y_test, 2)

## Creating the Model

In [9]:
classifier = Sequential()
classifier.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(230, 130, 1), padding="VALID"))
classifier.add(Dense(32, activation='relu'))
classifier.add(MaxPooling2D(pool_size=(3,3)))
classifier.add(Flatten())
classifier.add(Dense(2, activation='sigmoid'))

## Compiling the model

In [10]:
classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Changing Learnnig Rate

In [11]:
b.set_value(classifier.optimizer.learning_rate, 0.0001) 

## Setting up performance logging and parameter tracking

In [12]:
filepath = "/content/drive/MyDrive/Colab Notebooks/root_canal_classification/model_parameters/cnn7/cnn7-{epoch:02d}.hdfs" # Path to save checkpoints
checkpoint = ModelCheckpoint(filepath, save_freq=5) # checkpoint parameters

log_csv = CSVLogger("/content/drive/MyDrive/Colab Notebooks/root_canal_classification/model_logs/cnn7/cnn7.csv", separator=",", append=False) # Setting up csv logger for accuracy and loss tracking

callback_params = [checkpoint, log_csv]

## Fitting the model

In [13]:
classifier.fit(X_train, Y_train, epochs=2000, batch_size=32, validation_data=(X_test, Y_test), callbacks=callback_params) # Fitting the model

Streaming output truncated to the last 5000 lines.
9/9 [==============================] - 2s 238ms/step - loss: 3.7588e-04 - accuracy: 1.0000 - val_loss: 0.0295 - val_accuracy: 0.9855
Epoch 686/2000
9/9 [==============================] - 1s 130ms/step - loss: 3.4805e-04 - accuracy: 1.0000 - val_loss: 0.0231 - val_accuracy: 0.9855
Epoch 687/2000
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/root_canal_classification/model_parameters/cnn7/cnn7-687.hdfs/assets
9/9 [==============================] - 1s 113ms/step - loss: 3.6725e-04 - accuracy: 1.0000 - val_loss: 0.0260 - val_accuracy: 0.9855
Epoch 688/2000
9/9 [==============================] - 2s 224ms/step - loss: 3.5767e-04 - accuracy: 1.0000 - val_loss: 0.0249 - val_accuracy: 0.9855
Epoch 689/2000
9/9 [==============================] - 2s 229ms/step - loss: 3.4789e-04 - accuracy: 1.0000 - val_loss: 0.0254 - val_accuracy: 0.9855
Epoch 690/2000
9/9 [==============================] - 2s 223ms/step - loss: 3.768

## Saving Model

In [14]:
classifier.save("/content/drive/MyDrive/Colab Notebooks/root_canal_classification/model_config/cnn7")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/root_canal_classification/model_config/cnn7/assets
